In [ ]:
# @title 🎬 ГОДЖО 30 СЕК — С РЕАЛЬНОЙ АНИМАЦИЕЙ (AnimateDiff)
import os
import subprocess
import time
from IPython.display import FileLink, display

# ============================================
# 📝 НАСТРОЙКИ - МЕНЯЙ ТОЛЬКО ЭТО!
# ============================================
PROMPT = "gojo satoru, white spiky hair, black blindfold, confident smirk, turning head slowly, wind blowing hair, anime style, highly detailed, 8k, cinematic lighting, smooth animation"
NEGATIVE_PROMPT = "blurry, deformed, low quality, watermark, text, bad anatomy, static, frozen, choppy animation, jpeg artifacts"

# Параметры генерации
WIDTH = 512
HEIGHT = 768
NUM_FRAMES = 16  # AnimateDiff генерирует 16-24 кадра за раз
STEPS = 25  # Для анимации лучше больше steps
CFG_SCALE = 7.5
FPS = 8  # FPS исходной анимации

# Интерполяция RIFE (опционально - для сглаживания)
USE_RIFE = True  # True = применить RIFE для ещё более плавной анимации
RIFE_EXP = 4  # 4 = 16→256 кадров (~30 сек при 8 fps), 3 = 16→128 (~15 сек)
# ============================================

# === ПАПКИ ===
WORKSPACE = "/kaggle/working"
FRAMES_DIR = f"{WORKSPACE}/frames"
DATASET_DIR = "/kaggle/input/comfyui-models-gojo"

os.makedirs(FRAMES_DIR, exist_ok=True)

print("="*60)
print("🎬 ГЕНЕРАЦИЯ АНИМИРОВАННОГО ВИДЕО С AnimateDiff")
print("="*60)
print(f"Промпт: {PROMPT[:60]}...")
print(f"Размер: {WIDTH}x{HEIGHT}, {NUM_FRAMES} кадров")
print(f"RIFE интерполяция: {'Да' if USE_RIFE else 'Нет'}")
print("="*60 + "\n")

# === УСТАНОВКА ЗАВИСИМОСТЕЙ ===
print("🔧 Проверка зависимостей...")

try:
    import diffusers
    print("✓ diffusers")
except ImportError:
    print("Установка diffusers...")
    !pip install -q diffusers[torch] transformers accelerate

try:
    import cv2
    print("✓ opencv-python")
except ImportError:
    print("Установка opencv-python...")
    !pip install -q opencv-python

# Для AnimateDiff нужен imageio для сохранения видео
try:
    import imageio
    print("✓ imageio")
except ImportError:
    print("Установка imageio...")
    !pip install -q imageio imageio-ffmpeg

print("✓ Зависимости готовы!\n")

# === ГЕНЕРАЦИЯ АНИМАЦИИ С AnimateDiff ===
print("🎨 Загрузка AnimateDiff pipeline...")

import torch
from diffusers import AnimateDiffPipeline, MotionAdapter, EulerDiscreteScheduler
from diffusers.utils import export_to_gif, export_to_video
from PIL import Image

# Проверяем есть ли модель AnimateDiff в датасете
animatediff_model = f"{DATASET_DIR}/mm_sd_v15_v2.ckpt"
base_model = f"{DATASET_DIR}/sd_xl_base_1.0.safetensors"

if os.path.exists(animatediff_model):
    print(f"✓ Найдена AnimateDiff модель в датасете")
    # Для локальной модели используем другой подход
    # AnimateDiff работает с SD 1.5, а не SDXL
    use_local = False
else:
    print("Загрузка AnimateDiff из HuggingFace...")
    use_local = False

# AnimateDiff pipeline
if not use_local:
    # Используем motion adapter из HuggingFace
    adapter = MotionAdapter.from_pretrained(
        "guoyww/animatediff-motion-adapter-v1-5-2",
        torch_dtype=torch.float16
    )

    # Загружаем базовую SD 1.5 модель
    # AnimateDiff работает лучше с SD 1.5, чем с SDXL
    pipe = AnimateDiffPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        motion_adapter=adapter,
        torch_dtype=torch.float16
    ).to("cuda")

    # Оптимизация
    pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
    pipe.enable_vae_slicing()
    pipe.enable_model_cpu_offload()

print("✓ AnimateDiff pipeline готов!\n")

# === ГЕНЕРАЦИЯ АНИМАЦИИ ===
print(f"🎬 Генерация анимации {NUM_FRAMES} кадров...")
print(f"   Это займет ~3-7 минут в зависимости от GPU...\n")

start_time = time.time()

# Генерируем анимацию
output = pipe(
    prompt=PROMPT,
    negative_prompt=NEGATIVE_PROMPT,
    num_frames=NUM_FRAMES,
    width=WIDTH,
    height=HEIGHT,
    num_inference_steps=STEPS,
    guidance_scale=CFG_SCALE,
    generator=torch.Generator("cuda").manual_seed(42)
)

frames = output.frames[0]  # Список PIL изображений
gen_time = time.time() - start_time

print(f"✅ Анимация сгенерирована за {gen_time:.1f} секунд!")
print(f"   Получено {len(frames)} кадров\n")

# Сохраняем кадры для RIFE (если нужно)
if USE_RIFE:
    print("💾 Сохранение кадров для RIFE интерполяции...")
    for i, frame in enumerate(frames):
        frame.save(f"{FRAMES_DIR}/{i}.png")
    print(f"✓ {len(frames)} кадров сохранено\n")

# Создаем базовое видео из сгенерированной анимации
print("🎥 Создание базового видео...")
base_video_path = f"{WORKSPACE}/GOJO_ANIMATED_BASE.mp4"
export_to_video(frames, base_video_path, fps=FPS)

if os.path.exists(base_video_path):
    file_size = os.path.getsize(base_video_path) / 1024 / 1024
    duration = len(frames) / FPS
    print(f"✅ Базовое видео готово! ({duration:.1f}s, {file_size:.1f} MB)")
    print(f"   Скачать: ", end="")
    display(FileLink(base_video_path))
else:
    print("⚠️ Не удалось создать базовое видео")

# Очищаем память
del pipe
if 'adapter' in locals():
    del adapter
torch.cuda.empty_cache()

# === RIFE ИНТЕРПОЛЯЦИЯ (ОПЦИОНАЛЬНО) ===
final_video = base_video_path

if USE_RIFE and os.path.exists(base_video_path):
    print("\n" + "="*60)
    print(f"🎞️ RIFE интерполяция: {len(frames)} → {len(frames) * (2**RIFE_EXP)} кадров")
    print("="*60 + "\n")

    # Устанавливаем Practical-RIFE
    if not os.path.exists(f"{WORKSPACE}/RIFE"):
        print("Клонирование RIFE...")
        !git clone -q https://github.com/hzwer/Practical-RIFE {WORKSPACE}/RIFE

    os.chdir(f"{WORKSPACE}/RIFE")

    # Устанавливаем зависимости RIFE
    try:
        import skvideo
        print("✓ scikit-video")
    except ImportError:
        !pip install -q scikit-video

    # Скачиваем модель RIFE
    print("Скачивание модели RIFE v4.26...")

    if not os.path.exists("train_log/flownet.pkl"):
        !pip install -q gdown
        !mkdir -p train_log
        !gdown -q 1gViYvvQrtETBgU1w8axZSsr7YUuw31uy -O train_log.zip

        if os.path.exists("train_log.zip") and os.path.getsize("train_log.zip") > 1000000:
            !unzip -q train_log.zip

            # Ищем модель
            if not os.path.exists("train_log/flownet.pkl"):
                result = !find . -name "flownet.pkl" 2>/dev/null | head -1
                if result:
                    !cp {result[0]} train_log/flownet.pkl

    model_downloaded = os.path.exists("train_log/flownet.pkl")

    if model_downloaded:
        print("✓ Модель RIFE готова!\n")
        print(f"Запуск интерполяции (exp={RIFE_EXP})...")
        print("Это займет 5-15 минут...\n")

        rife_start = time.time()

        # Запускаем RIFE
        !python inference_video.py \
            --img {FRAMES_DIR} \
            --exp {RIFE_EXP} \
            --output {WORKSPACE}/GOJO_ANIMATED_RIFE.mp4 \
            --UHD \
            2>/dev/null

        rife_time = time.time() - rife_start

        # Проверяем результат
        rife_video = f"{WORKSPACE}/GOJO_ANIMATED_RIFE.mp4"
        if os.path.exists(rife_video):
            file_size = os.path.getsize(rife_video) / 1024 / 1024
            final_frames = len(frames) * (2**RIFE_EXP)
            duration = final_frames / (FPS * (2**RIFE_EXP))

            print(f"\n✅ RIFE интерполяция завершена за {rife_time:.1f}s!")
            print(f"   {len(frames)} → {final_frames} кадров")
            print(f"   Размер: {file_size:.1f} MB")
            final_video = rife_video
        else:
            print("\n⚠️ RIFE не создал видео, используем базовую анимацию")
    else:
        print("⚠️ Не удалось скачать модель RIFE")
        print("Используем базовую анимацию без интерполяции")

# === РЕЗУЛЬТАТ ===
print("\n" + "="*60)
print("🎉 ГОТОВО!")
print("="*60)
print(f"📋 Итоги:")
print(f"   Промпт: {PROMPT[:50]}...")
print(f"   Размер: {WIDTH}x{HEIGHT}")
print(f"   Время генерации: {gen_time:.1f}s")

if USE_RIFE and 'rife_time' in locals():
    print(f"   Время RIFE: {rife_time:.1f}s")
    print(f"   Общее время: {gen_time + rife_time:.1f}s")

print(f"\n📥 ФИНАЛЬНОЕ ВИДЕО:")
display(FileLink(final_video))

if os.path.exists(base_video_path) and final_video != base_video_path:
    print(f"\n📥 Базовая анимация (без RIFE):")
    display(FileLink(base_video_path))

print("\n" + "="*60)
print("💡 СОВЕТЫ:")
print("   • Добавьте в промпт описание движения:")
print("     'turning head', 'blinking', 'hair flowing', 'camera zoom'")
print("   • Для более плавной анимации увеличьте NUM_FRAMES до 24")
print("   • Используйте 'smooth motion' в промпте")
print("   • Negative промпт: 'static, frozen, choppy'")
print("="*60)
